# Topological K-means



#### Imports

In [ ]:
import sys
import time
import warnings

import networkx as nx
import matplotlib.pyplot as plt
import scipy.sparse._csr
import numpy as np
from numpy import dot
from numpy import trace
from numpy.linalg import inv

import sklearn.datasets as skdata
import sklearn.neighbors as sknn
import sklearn.utils.graph as sksp
from sklearn import preprocessing
from sklearn.cluster import KMeans
from sklearn.metrics.cluster import rand_score
from sklearn.metrics.cluster import silhouette_score

# To avoid unnecessary warning messages
warnings.simplefilter(action='ignore')

#### Implementation

In [ ]:
# Implementação própria do algoritmo K-means geodésico
def GeodesicKmeans(dados, nn):
    print('Executando Geodesic K-means...')
    iteracao = 0

    # Número de amostras, atributos e clusters
    n = dados.shape[0]
    m = dados.shape[1]
    k = len(np.unique(target))

    # Escolhe k observações aleatoriamente
    coord_centros = np.random.choice(n, size=k, replace=False)
    centros = dados[coord_centros]

    # Adiciona centros no final dos dados (k úlimas linhas)
    dados = np.vstack((dados, centros))

    # Loop principal (repete passos anteriores)
    while True:
        iteracao += 1
        #print(iteracao)
        # Gera o grafo KNN
        knnGraph = sknn.kneighbors_graph(dados, n_neighbors=nn, mode='distance')
        # Extrai a matriz de adjacências ponderada a partir do grafo KNN
        W = knnGraph.toarray()
        # Calcula as distâncias geodésicas
        G = nx.from_numpy_array(W)
        D = nx.floyd_warshall_numpy(G)
        # Pega distâncias de cada centro a todos os demais pontos
        # Coordenadas dos centros são k últimas linhas de dados
        if iteracao > 0:
            coord_centros = list(range(n, n+k))
        distancias = D[coord_centros, :]
        # Cria vetor de rótulos
        rotulos = np.zeros(n)
        # Atualiza os rótulos (menor distância)
        for j in range(n):
            rotulos[j] = distancias[:, j].argmin()
        # Atualiza os centros
        novos_centros = np.zeros((k, m))
        for r in range(k):
            indices = np.where(rotulos==r)[0]
            #print(inds)
            if len(indices) > 0:
                sample = dados[indices]
                novos_centros[r, :] = sample.mean(axis=0)
            else:
                novos_centros[r, :] = centros[r, :]
        # Atualiza centros no final dos dados (k úlimas linhas)
        dados[n:n+k, :] = novos_centros
        # Condição de parada
        if (np.linalg.norm(centros - novos_centros) < 0.1) or iteracao > 10:
            break
        # Atualiza centros
        centros = novos_centros.copy()

    return rotulos

#### Reading the data

In [ ]:
# Scikit-learn datasets
X = skdata.load_iris()
#X = skdata.fetch_openml(name='segment', version=2)
#X = skdata.fetch_openml(name='amazon-commerce-reviews', version=1)
#X = skdata.fetch_openml(name='semeion', version=1)
#X = skdata.fetch_openml(name='mfeat-pixel', version=3)
#X = skdata.fetch_openml(name='AP_Colon_Kidney', version=1)      # Excelente!
#X = skdata.fetch_openml(name='penguins', version=1)
#X = skdata.fetch_openml(name='cnae-9', version=2)
#X = skdata.fetch_openml(name='micro-mass', version=1)
#X = skdata.fetch_openml(name='oh5.wc', version=1)
#X = skdata.fetch_openml(name='tr45.wc', version=1)
#X = skdata.fetch_openml(name='GCM', version=1)
#X = skdata.fetch_openml(name='leukemia', version=1)              # Excelente!
#X = skdata.fetch_openml(name='AP_Colon_Prostate', version=1)     # Excelente!
#X = skdata.fetch_openml(name='AP_Breast_Lung', version=1)        # Muito bom!
#X = skdata.fetch_openml(name='cloud', version=2)
##X = skdata.fetch_openml(name='collins', version=2)
##X = skdata.fetch_openml(name='pyrim', version=2)
##X = skdata.fetch_openml(name='balance-scale', version=1)
##X = skdata.fetch_openml(name='servo', version=1)
##X = skdata.fetch_openml(name='monks-problems-1', version=1)
##X = skdata.fetch_openml(name='breast-tissue', version=1)
##X = skdata.fetch_openml(name='fri_c2_100_10', version=2)
##X = skdata.fetch_openml(name='datatrieve', version=1)
##X = skdata.fetch_openml(name='fri_c3_250_25', version=2)

# Lentos
#X = skdata.load_digits()
#X = skdata.fetch_openml(name='phoneme', version=1)
#X = skdata.fetch_openml(name='Bioresponse', version=1)
#X = skdata.fetch_openml(name='optdigits', version=1)
#X = skdata.fetch_openml(name='satimage', version=1)

dados = X['data']
target = X['target']

n = dados.shape[0]
m = dados.shape[1]
c = len(np.unique(target))
nn = round(np.sqrt(n))

print('N = ', n)
print('M = ', m)
print('C = %d' %c)
print()

# Matriz esparsa (em alguns datasets com dimensionalidade muito alta)
if type(dados) == scipy.sparse._csr.csr_matrix:
    dados = dados.todense()
    dados = np.asarray(dados)
else:
    # Alguns datasets contém atributos categóricos (texto). Converte para valores inteiros.
    if not isinstance(dados, np.ndarray):
        cat_cols = dados.select_dtypes(['category']).columns
        dados[cat_cols] = dados[cat_cols].apply(lambda x: x.cat.codes)
        # Convert to numpy
        dados = dados.to_numpy()
        target = target.to_numpy()

# Transforma rótulos para inteiros
rotulos = list(np.unique(target))
numbers = np.zeros(n)
for i in range(n):
    numbers[i] = rotulos.index(target[i])

N =  150
M =  4
C = 3



#### Geodesic K-means

In [ ]:

# Executa o k-means diversas vezes
inicio = time.time()

MAX = 31
lista_rand = []
lista_sc = []
# Executa 30 vezes pois depende da inicialização aleatória
for i in range(1, MAX):
    # Número de vizinhos no grafo deve ser suficientemente grande para que grafo seja conexo
    labels = GeodesicKmeans(dados, nn)
    # External indices
    lista_rand.append(rand_score(target, labels))
    # Internal indices
    lista_sc.append(silhouette_score(dados, labels))
fim = time.time()

print()
print('GEODESIC K-MEANS')
print('Elapsed time: %.4f' %(fim - inicio))
print()
print('Maximum Rand index: %.4f' %(max(lista_rand)))
print('Maximum Calinski Harabasz index: %.4f' %(max(lista_sc)))
print(labels)
print(target)

Executando Geodesic K-means...
Graph with 153 nodes and 1181 edges
Graph with 153 nodes and 1194 edges
Executando Geodesic K-means...
Graph with 153 nodes and 1185 edges
Graph with 153 nodes and 1197 edges
Graph with 153 nodes and 1191 edges
Executando Geodesic K-means...
Graph with 153 nodes and 1180 edges
Graph with 153 nodes and 1186 edges
Graph with 153 nodes and 1192 edges
Graph with 153 nodes and 1192 edges
Executando Geodesic K-means...
Graph with 153 nodes and 1176 edges
Graph with 153 nodes and 1194 edges
Graph with 153 nodes and 1193 edges
Executando Geodesic K-means...
Graph with 153 nodes and 1175 edges
Graph with 153 nodes and 1182 edges
Graph with 153 nodes and 1186 edges
Graph with 153 nodes and 1190 edges
Graph with 153 nodes and 1190 edges
Executando Geodesic K-means...
Graph with 153 nodes and 1180 edges
Graph with 153 nodes and 1187 edges
Graph with 153 nodes and 1191 edges
Executando Geodesic K-means...
Graph with 153 nodes and 1183 edges
Graph with 153 nodes and 11

#### Classical K-means

In [ ]:

# Executa o k-means diversas vezes
inicio = time.time()

MAX = 31
lista_rand = []
lista_sc = []

# Executa 30 execuções do k-médias padrão e toma melhor desempenho
for i in range(1, MAX):
    kmeans = KMeans(n_clusters=c, init='random', n_init=1).fit(dados)
    # External indices
    lista_rand.append(rand_score(target, kmeans.labels_))
    # Internal indices
    lista_sc.append(silhouette_score(dados, kmeans.labels_))
fim = time.time()


print('K-MEANS padrão')
print('Elapsed time: %.4f' %(fim - inicio))
print()
print('Maximum Rand index: %.4f' %(max(lista_rand)))
print('Maximum Silhouette coefficient: %.4f' %(max(lista_sc)))

print(kmeans.labels_)
print(target)

K-MEANS padrão
Elapsed time: 0.4211

Maximum Rand index: 0.8797
Maximum Silhouette coefficient: 0.5528
[0 0 0 0 0 1 0 0 0 0 1 0 0 0 1 1 1 0 1 1 1 1 0 1 0 0 0 1 0 0 0 1 1 1 0 0 1
 0 0 0 0 0 0 1 1 0 1 0 1 0 2 2 2 2 2 2 2 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 1 2 2 2 2 1 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2]
